In [2]:
import tensorflow as tf

In [5]:
graph = tf.Graph()
 
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, None, 9],
        name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, 2], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

In [ ]:
model = Sequential()
model.add(LSTM(input_dim=1, output_dim=16,  return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(32,return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(output_dim=1))
model.add(Activation("linear"))

model.compile(loss="mse", optimizer="rmsprop")

model.fit(X_train, y_train, batch_size=512, nb_epoch=epochs, validation_split=0.40)